In [9]:
# Expanded to cover all variants in PROFILE_GEOMETRY_MAP
STEEL_DB_MAP = {
    # Parallel Flange I-Sections (Standard, Light, Heavy, Extra)
    "sections_ipe": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_ipea": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_ipeaa": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_ipeo": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_hea": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_heaa": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_heb": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_hem": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_he": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_hd": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_hl": {"function": "create_i_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},

    # Tapered I-Sections
    "sections_ipn": {"function": "create_ipn_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root", "r_toe": "r_toe"}},

    # Channels (Parallel)
    "sections_upe": {"function": "create_u_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},
    "sections_uap": {"function": "create_u_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root"}},

    # Channels (Tapered)
    "sections_upn": {"function": "create_upn_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root", "r_toe": "r_toe"}},
    "sections_ue": {"function": "create_ue_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root", "r_toe": "r_toe"}},

    # Angles (Unequal, Equal, and variants)
    "sections_lu": {"function": "create_angle_geometry", "params": {"h": "h", "b": "b", "t": "t", "r_root": "r_root", "r_toe": "r_toe"}},
    "sections_le": {"function": "create_angle_geometry", "params": {"h": "b", "b": "b", "t": "t", "r_root": "r_root", "r_toe": "r_toe"}},

    # Hollow Sections
    "sections_rhs": {"function": "create_rhs_geometry", "params": {"h": "h", "b": "b", "t": "t", "r_out": "r_out", "r_in": "r_in"}},
    "sections_shs": {"function": "create_shs_geometry", "params": {"a": "h", "t": "t", "r_out": "r_out", "r_in": "r_in"}},
    "sections_chs": {"function": "create_chs_geometry", "params": {"d": "D", "t": "T"}},

    # T-Sections
    "sections_t": {"function": "create_t_section_geometry", "params": {"h": "h", "b": "b", "tf": "tf", "tw": "tw", "r_root": "r_root", "r_toe": "r_toe", "r_web": "r_web"}}
}

In [10]:
all_results = []
print("🚀 Starting Full Library Validation...")

conn = sqlite3.connect(db_path)

for table_name, config in STEEL_DB_MAP.items():
    try:
        # Check table existence
        check = pd.read_sql(
            f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'", conn)
        if check.empty:
            print(f"⏩ Skipping {table_name} (Not found)")
            continue

        df_table = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        print(f"📦 Processing {table_name:.<25} ({len(df_table)} profiles)")

        geom_func = getattr(dfun, config["function"])

        for _, row in df_table.iterrows():
            func_args = {arg: (float(row[col]) if pd.notna(row.get(col)) else 0.0)
                         for arg, col in config["params"].items()}

            # Pass section_id if function supports it
            func_args['section_id'] = row.get('Section_ID', 'Unknown')

            try:
                geom = geom_func(**func_args)
                db_area = float(row['A'])
                calc_area = geom.area
                diff = (abs(calc_area - db_area) /
                        db_area * 100) if db_area > 0 else 0

                all_results.append({
                    "Family": table_name.replace("sections_", "").upper(),
                    "ID": row['Section_ID'],
                    "DB_Area": db_area,
                    "Calc_Area": calc_area,
                    "Error": diff
                })
            except Exception:
                continue

    except Exception as e:
        print(f"❌ Error in {table_name}: {e}")

conn.close()

df_master = pd.DataFrame(all_results)

# Final Reporting
report = df_master.groupby("Family")["Error"].agg(
    ['count', 'mean', 'max']).reset_index()
report.columns = ['Family', 'Count', 'Mean Error (%)', 'Max Error (%)']
report = report.sort_values("Mean Error (%)")

print("\n" + "="*65)
print("🏆 FINAL GLOBAL ACCURACY REPORT (FULL COVERAGE)")
print("="*65)
print(report.to_string(index=False))
print("-" * 65)
print(f"TOTAL PROFILES: {len(df_master)}")
print(f"OVERALL MEAN ERROR: {df_master['Error'].mean():.6f}%")
print("="*65)

🚀 Starting Full Library Validation...
📦 Processing sections_ipe............. (18 profiles)
📦 Processing sections_ipea............ (18 profiles)
📦 Processing sections_ipeaa........... (9 profiles)
📦 Processing sections_ipeo............ (17 profiles)
📦 Processing sections_hea............. (24 profiles)
📦 Processing sections_heaa............ (24 profiles)
📦 Processing sections_heb............. (24 profiles)
📦 Processing sections_hem............. (24 profiles)
📦 Processing sections_he.............. (41 profiles)
📦 Processing sections_hd.............. (42 profiles)
📦 Processing sections_hl.............. (39 profiles)
📦 Processing sections_ipn............. (21 profiles)
📦 Processing sections_upe............. (14 profiles)
📦 Processing sections_uap............. (9 profiles)
📦 Processing sections_upn............. (18 profiles)
📦 Processing sections_ue.............. (13 profiles)
📦 Processing sections_lu.............. (88 profiles)
📦 Processing sections_le.............. (134 profiles)
📦 Process